In [ ]:
input_file = 'input.txt'
lines = open(input_file, 'r').readlines()
lines = [line.strip() for line in lines]

In [ ]:
import networkx as nx

In [ ]:
def make_id_point(point):
    return 1000 * point[0] +  point[1]

def make_label_point(point):
    res = [{'#':'Tree', '.':'Path', '>':'Slope_E','<':'Slope_W','^':'Slope_N','v':'Slope_S'}[lines[point[1]][point[0]]]]
    if point[1] == 0 and lines[point[1]][point[0]] == '.':
        res += ['Source']
    if point[1] == height-1 and lines[point[1]][point[0]] == '.':
        res += ['Target']
    return res

In [ ]:
maze = nx.DiGraph()
height = len(lines)
width = len(lines[0])
for j, _ in enumerate(lines):
    for i, _ in enumerate(lines[0]):
        maze.add_node(make_id_point((i,j)), labels=make_label_point((i,j)) + ['Point'], i=i, j=j)
        if not lines[j][i] in ['#']:
            maze.add_edge(make_id_point((i,j)), make_id_point((i,j)) , relationshipType='VISIT')
        if i>0 and lines[j][i] in ['.','<'] and not lines[j][i-1] in ['#']:
            maze.add_edge(make_id_point((i,j)), make_id_point((i-1,j)) , relationshipType='MOVE', dirx=-1, diry=0)
        if i<width-1 and lines[j][i] in ['.','>'] and not lines[j][i+1] in ['#']:
            maze.add_edge(make_id_point((i,j)), make_id_point((i+1,j)) , relationshipType='MOVE', dirx=1, diry=0)
        if j>0 and lines[j][i] in ['.','^'] and not lines[j-1][i] in ['#']:
            maze.add_edge(make_id_point((i,j)), make_id_point((i,j-1)) , relationshipType='MOVE', dirx=0, diry=-1)
        if j<height-1 and lines[j][i] in ['.','v'] and not lines[j+1][i] in ['#']:
            maze.add_edge(make_id_point((i,j)), make_id_point((i,j+1)) , relationshipType='MOVE', dirx=0, diry=1)

In [ ]:
NEO4J_URI = "neo4j://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "pierre!!"
from graphdatascience import GraphDataScience

In [ ]:
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD), aura_ds=False, database="neo4j")


In [ ]:
G = gds.graph.networkx.load(maze, "maze")


In [ ]:
gds.graph.export(G, dbName="mazepersisted")


In [ ]:
gds.run_cypher('CREATE DATABASE mazepersisted')


In [ ]:
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD), aura_ds=True, database="mazepersisted")

In [ ]:
gds.run_cypher("""
MATCH p=(s:Source)
(()-[:VISIT]->()-[m:MOVE]->(v))+
(t:Target)
WITH m
CALL {
    WITH m
    WITH size(m) AS length
    RETURN max(length) AS length
} IN TRANSACTIONS OF 10 ROWS
RETURN max(length) AS part1
""")

# Part 2

In [ ]:
def make_id_point(point):
    return 1000 * point[0] +  point[1]

def make_label_point(point):
    res = [{'#':'Tree', '.':'Path', '>':'Path','<':'Path','^':'Path','v':'Path'}[lines[point[1]][point[0]]]]
    if point[1] == 0 and lines[point[1]][point[0]] == '.':
        res += ['Source']
    if point[1] == height-1 and lines[point[1]][point[0]] == '.':
        res += ['Target']
    return res

In [ ]:
maze = nx.DiGraph()
height = len(lines)
width = len(lines[0])
for j, _ in enumerate(lines):
    for i, _ in enumerate(lines[0]):
        maze.add_node(make_id_point((i,j)), labels=make_label_point((i,j)) + ['Point'], i=i, j=j)
        if not lines[j][i] in ['#']:
            maze.add_edge(make_id_point((i,j)), make_id_point((i,j)) , relationshipType='VISIT')
        if i>0 and lines[j][i] in ['.','<', '>', '^', 'v'] and not lines[j][i-1] in ['#']:
            maze.add_edge(make_id_point((i,j)), make_id_point((i-1,j)) , relationshipType='MOVE', dirx=-1, diry=0)
        if i<width-1 and lines[j][i] in ['.','<', '>', '^', 'v'] and not lines[j][i+1] in ['#']:
            maze.add_edge(make_id_point((i,j)), make_id_point((i+1,j)) , relationshipType='MOVE', dirx=1, diry=0)
        if j>0 and lines[j][i] in ['.','<', '>', '^', 'v'] and not lines[j-1][i] in ['#']:
            maze.add_edge(make_id_point((i,j)), make_id_point((i,j-1)) , relationshipType='MOVE', dirx=0, diry=-1)
        if j<height-1 and lines[j][i] in ['.','<', '>', '^', 'v'] and not lines[j+1][i] in ['#']:
            maze.add_edge(make_id_point((i,j)), make_id_point((i,j+1)) , relationshipType='MOVE', dirx=0, diry=1)

In [ ]:
NEO4J_URI = "neo4j://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "pierre!!"
from graphdatascience import GraphDataScience

In [ ]:
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

In [ ]:
G = gds.graph.networkx.load(maze, "climbmaze")

gds.graph.export(G, dbName="climbmazepersisted")


In [ ]:
gds.run_cypher('CREATE DATABASE climbmazepersisted')


In [ ]:
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD), database="climbmazepersisted")

In [ ]:
gds.run_cypher("""
CYPHER runtime=parallel
MATCH p=(s:Source)
(()-[:VISIT]->()-[m:MOVE]->(v))+
(t:Target)
WITH m
CALL {
    WITH m
    WITH size(m) AS length
    RETURN max(length) AS length
} IN TRANSACTIONS OF 10 ROWS
RETURN max(length) AS part2
""")